### 1. 패키지 불러오기

In [1]:
import gc
import numpy as np
import pandas as pd
import lightgbm as lgb
from  datetime import datetime, timedelta

### 2. 데이터셋 전처리

### 2-1. CALENDAR

In [2]:
path= 'C:/Users/User/Desktop/m5-forecasting-accuracy/'

# Correct data types for "calendar.csv"
calendarDTypes = {"event_name_1": "category", 
                  "event_name_2": "category", 
                  "event_type_1": "category", 
                  "event_type_2": "category", 
                  "weekday": "category", 
                  'wm_yr_wk': 'int16', 
                  "wday": "int16",
                  "month": "int16", 
                  "year": "int16", 
                  "snap_CA": "float32", 
                  'snap_TX': 'float32', 
                  'snap_WI': 'float32' }

# Read csv file
calendar = pd.read_csv(path+"calendar.csv", 
                       dtype = calendarDTypes)

calendar["date"] = pd.to_datetime(calendar["date"])

# Transform categorical features into integers
for col, colDType in calendarDTypes.items():
    if colDType == "category":
        calendar[col] = calendar[col].cat.codes.astype("int16")
        calendar[col] -= calendar[col].min()

### 2-2. SELL PRICE

In [3]:
# Correct data types for "sell_prices.csv"
priceDTypes = {"store_id": "category", 
               "item_id": "category", 
               "wm_yr_wk": "int16",
               "sell_price":"float32"}

# Read csv file
prices = pd.read_csv(path+"sell_prices.csv", dtype = priceDTypes)

# Transform categorical features into integers
for col, colDType in priceDTypes.items():
    if colDType == "category":
        prices[col] = prices[col].cat.codes.astype("int16")
        prices[col] -= prices[col].min()
        
prices.head()

,store_id,item_id,wm_yr_wk,sell_price
0,0,0,11325,9.58
1,0,0,11326,9.58
2,0,0,11327,8.26
3,0,0,11328,8.26
4,0,0,11329,8.26


### 2-3. SALES_TRAIN_VALIDATION

In [36]:
firstDay = 250
lastDay = 1913

# Use x sales days (columns) for training
numCols = [f"d_{day}" for day in range(firstDay, lastDay+1)]

# Define all categorical columns
catCols = ['id', 'item_id', 'dept_id','store_id', 'cat_id', 'state_id']

# Define the correct data types for "sales_train_validation.csv"
dtype = {numCol: "float32" for numCol in numCols} 
dtype.update({catCol: "category" for catCol in catCols if catCol != "id"})

# Read csv file
ds = pd.read_csv(path+"sales_train_validation.csv", 
                 usecols = catCols + numCols, dtype = dtype)

# Transform categorical features into integers
for col in catCols:
    if col != "id":
        ds[col] = ds[col].cat.codes.astype("int16")
        ds[col] -= ds[col].min()
        
ds = pd.melt(ds,
             id_vars = catCols,
             value_vars = [col for col in ds.columns if col.startswith("d_")],
             var_name = "d",
             value_name = "sales")

# Merge "ds" with "calendar" and "prices" dataframe
ds = ds.merge(calendar, on = "d", copy = False)
ds = ds.merge(prices, on = ["store_id", "item_id", "wm_yr_wk"], copy = False)

ds.head()

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,...,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_002_CA_1_validation,1,0,0,0,0,d_250,0.0,2011-10-05,11136,...,10,2011,0,0,0,0,1.0,1.0,1.0,3.97
1,HOBBIES_1_002_CA_1_validation,1,0,0,0,0,d_251,0.0,2011-10-06,11136,...,10,2011,0,0,0,0,1.0,1.0,1.0,3.97
2,HOBBIES_1_002_CA_1_validation,1,0,0,0,0,d_252,0.0,2011-10-07,11136,...,10,2011,0,0,0,0,1.0,1.0,0.0,3.97
3,HOBBIES_1_004_CA_1_validation,3,0,0,0,0,d_250,0.0,2011-10-05,11136,...,10,2011,0,0,0,0,1.0,1.0,1.0,4.34
4,HOBBIES_1_004_CA_1_validation,3,0,0,0,0,d_251,4.0,2011-10-06,11136,...,10,2011,0,0,0,0,1.0,1.0,1.0,4.34


### 3. 피쳐 생성

In [37]:
ds_copy= ds.copy()

### 3-1. SALES FEATURE

dayLags = [7, 28]
lagSalesCols = [f"lag_{dayLag}" for dayLag in dayLags]
for dayLag, lagSalesCol in zip(dayLags, lagSalesCols):
    ds_copy[lagSalesCol] = ds_copy[["id","sales"]].groupby("id")["sales"].shift(dayLag)
    
windows = [7, 28]
for window in windows:
    for dayLag, lagSalesCol in zip(dayLags, lagSalesCols):
        ds_copy[f"rmean_{dayLag}_{window}"] = ds_copy[["id", lagSalesCol]].groupby("id")[lagSalesCol].transform(lambda x: x.rolling(window).mean())

In [39]:
%%time

# 왜 Shift를 해야할까?
# Shift 28을 하지 않으면 예측값이 뒤로 가면갈 수록  F1->F28 예측 할 수 있는 변수가 줄어든게 된다.
# 28일은 한달을 의미한다. 최근 한달간의 경향성을 보는 것으로 보면 되겠다.
# 28일을 56일로 늘리면 안되나? - 최근 한달간의 경향성이 반영이 안되는 것이낙?
# https://www.kaggle.com/kneroma/m5-first-public-notebook-under-0-50


ds['sales_1'] = ds[['id','sales']].groupby("id")['sales'].shift(1)
# ds['sales_2'] = ds[['id','sales']].groupby("id")['sales'].shift(2)
# ds['sales_3'] = ds[['id','sales']].groupby("id")['sales'].shift(3)


ds['sales_7'] = ds[['id','sales']].groupby("id")['sales'].shift(7)
ds['sales_28'] = ds[['id','sales']].groupby("id")['sales'].shift(28)

print("mean")

ds['rmean_1_7'] = ds[['id','sales_1']].groupby("id")['sales_1'].transform(lambda x: x.rolling(14).mean())
ds['rmean_7_7'] = ds[['id','sales_7']].groupby("id")['sales_7'].transform(lambda x: x.rolling(7).mean())
ds['rmean_7_28'] = ds[['id','sales_7']].groupby("id")['sales_7'].transform(lambda x: x.rolling(28).mean())
ds['rmean_7_50'] = ds[['id','sales_7']].groupby("id")['sales_7'].transform(lambda x: x.rolling(50).mean())

ds['rmean_28_7'] = ds[['id','sales_28']].groupby("id")['sales_28'].transform(lambda x: x.rolling(7).mean())
ds['rmean_28_28'] = ds[['id','sales_28']].groupby("id")['sales_28'].transform(lambda x: x.rolling(28).mean())
ds['rmean_28_50'] = ds[['id','sales_28']].groupby("id")['sales_28'].transform(lambda x: x.rolling(50).mean())
ds['rmean_1_365'] = ds[['id','sales_1']].groupby("id")['sales_1'].transform(lambda x: x.rolling(365).mean())

# # print("std")
# # ds['rstd_7'] = ds[['id','sales']].groupby("id")['sales'].transform(lambda x: x.rolling(7 ,min_periods=1).std())
# # ds['rstd_28'] = ds[['id','sales']].groupby("id")['sales'].transform(lambda x: x.rolling(28 ,min_periods=1).std())
ds['rstd_365'] = ds[['id','sales']].groupby("id")['sales'].transform(lambda x: x.rolling(365 ,min_periods=1).std())

# # print("max")
# # ds['rmax_7'] = ds[['id','sales']].groupby("id")['sales'].transform(lambda x: x.rolling(7 ,min_periods=1).max())
# # ds['rmax_28'] = ds[['id','sales']].groupby("id")['sales'].transform(lambda x: x.rolling(28 ,min_periods=1).max())
# # ds['rmax_50'] = ds[['id','sales']].groupby("id")['sales'].transform(lambda x: x.rolling(50 ,min_periods=1).max())

# # print("min")
# # ds['rmin_7'] = ds[['id','sales']].groupby("id")['sales'].transform(lambda x: x.rolling(7 ,min_periods=1).min())
# # ds['rmin_28'] = ds[['id','sales']].groupby("id")['sales'].transform(lambda x: x.rolling(28 ,min_periods=1).min())
# # ds['rmin_50'] = ds[['id','sales']].groupby("id")['sales'].transform(lambda x: x.rolling(50 ,min_periods=1).min())

# # print("count")
#ds['rcnt_60'] = ds[['id','sales']].groupby("id")['sales'].transform(lambda x: x.rolling(60).count() if x>0).fllna(0)
#ds['rcnt_180'] = ds[['id','sales']].groupby("id")['sales'].transform(lambda x: x.rolling(180).count() if x>0).fllna(0)
#ds['rcnt_365'] = ds[['id','sales']].groupby("id")['sales'].transform(lambda x: x.rolling(365).count() if x>0).fllna(0)

mean
Wall time: 4min 34s


### 3-2. DATE FEATURES

In [22]:
dateFeatures = {"wday": "weekday",
                "week": "weekofyear",
                "month": "month",
                "quarter": "quarter",
                "year": "year",
                "mday": "day"}

for featName, featFunc in dateFeatures.items():
    if featName in ds.columns:
        ds[featName] = ds[featName].astype("int16")
    else:
        ds[featName] = getattr(ds["date"].dt, featFunc).astype("int16")

In [25]:
ds.tail(50)

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,...,rmean_28_28,week,quarter,mday,sales_1,sales_7,sales_28,rmean_1_7,rmean_7_50,rmean_28_50
42372632,FOODS_3_803_WI_3_validation,3024,6,9,2,2,d_1912,0.0,2016-04-23,11613,...,1.607143,16,2,23,0.0,0.0,0.0,0.000000,1.100000,1.180000
42372633,FOODS_3_803_WI_3_validation,3024,6,9,2,2,d_1913,0.0,2016-04-24,11613,...,1.571429,16,2,24,0.0,0.0,0.0,0.000000,1.060000,1.160000
42372634,FOODS_3_804_WI_3_validation,3025,6,9,2,2,d_1912,4.0,2016-04-23,11613,...,9.785714,16,2,23,0.0,12.0,11.0,6.000000,8.480000,7.780000
42372635,FOODS_3_804_WI_3_validation,3025,6,9,2,2,d_1913,2.0,2016-04-24,11613,...,10.250000,16,2,24,4.0,3.0,17.0,6.142857,8.500000,8.040000
42372636,FOODS_3_805_WI_3_validation,3026,6,9,2,2,d_1912,5.0,2016-04-23,11613,...,1.785714,16,2,23,0.0,0.0,2.0,1.785714,1.740000,2.240000
42372637,FOODS_3_805_WI_3_validation,3026,6,9,2,2,d_1913,1.0,2016-04-24,11613,...,1.750000,16,2,24,5.0,4.0,1.0,1.928571,1.780000,2.200000
42372638,FOODS_3_806_WI_3_validation,3027,6,9,2,2,d_1912,0.0,2016-04-23,11613,...,0.392857,16,2,23,0.0,3.0,1.0,0.857143,0.660000,0.240000
42372639,FOODS_3_806_WI_3_validation,3027,6,9,2,2,d_1913,1.0,2016-04-24,11613,...,0.464286,16,2,24,0.0,1.0,2.0,0.714286,0.660000,0.280000
42372640,FOODS_3_807_WI_3_validation,3028,6,9,2,2,d_1912,3.0,2016-04-23,11613,...,3.000000,16,2,23,0.0,5.0,9.0,3.642857,3.800000,2.640000
42372641,FOODS_3_807_WI_3_validation,3028,6,9,2,2,d_1913,0.0,2016-04-24,11613,...,3.071429,16,2,24,3.0,4.0,7.0,3.500000,3.840000,2.680000


In [40]:
ds_zero= ds[ds['id'].isin(zero_list)]

In [41]:
ds_zero2= ds_zero[ds_zero['date']==ds_zero['date'].max()]

In [42]:
ds_zero2.to_csv("test.csv")

In [68]:
test= ds[ds['date']==ds['date'].max()]
#test= test[~test['id'].isin(zero_list)]
#test.to_csv("test3.csv")

In [69]:
test['sigma']=test['rmean_1_365']/test['rstd_365']

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [127]:
test2= test[test['sigma']<0.4]
test3= test2[(test2['sigma']<test2['rmean_1_7'])&(test2['sigma']<test2['rmean_1_7'])]

In [132]:
zero_fnl= test3['id'].tolist()

In [131]:
test3

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,...,rmean_1_7,rmean_7_7,rmean_7_28,rmean_7_50,rmean_28_7,rmean_28_28,rmean_28_50,rmean_1_365,rstd_365,sigma
42311719,HOBBIES_1_009_CA_1_validation,8,0,0,0,0,d_1913,0.0,2016-04-24,11613,...,1.142857,1.142857,0.642857,0.54,0.000000,0.321429,0.18,0.238356,0.828306,0.287763
42311753,HOBBIES_1_026_CA_1_validation,25,0,0,0,0,d_1913,0.0,2016-04-24,11613,...,0.285714,0.428571,0.285714,0.24,0.285714,0.214286,0.20,0.046575,0.211017,0.220719
42311807,HOBBIES_1_053_CA_1_validation,52,0,0,0,0,d_1913,0.0,2016-04-24,11613,...,0.428571,0.857143,0.214286,0.16,0.000000,0.071429,0.08,0.120548,0.510053,0.236344
42311831,HOBBIES_1_066_CA_1_validation,64,0,0,0,0,d_1913,0.0,2016-04-24,11613,...,1.071429,1.285714,0.392857,0.82,0.000000,0.964286,0.98,0.808219,2.045626,0.395096
42311857,HOBBIES_1_080_CA_1_validation,77,0,0,0,0,d_1913,7.0,2016-04-24,11613,...,1.000000,0.000000,0.142857,0.64,0.428571,1.107143,1.24,1.060274,2.663704,0.398045
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42372543,FOODS_3_758_WI_3_validation,2979,6,9,2,2,d_1913,0.0,2016-04-24,11613,...,0.785714,1.285714,0.321429,0.36,0.000000,0.321429,0.24,0.115068,0.466360,0.246737
42372551,FOODS_3_762_WI_3_validation,2983,6,9,2,2,d_1913,0.0,2016-04-24,11613,...,0.571429,0.714286,0.964286,1.30,1.428571,1.678571,1.28,0.230137,0.664377,0.346395
42372607,FOODS_3_790_WI_3_validation,3011,6,9,2,2,d_1913,0.0,2016-04-24,11613,...,0.142857,0.142857,0.357143,0.20,0.000000,0.000000,0.00,0.030137,0.342376,0.088023
42372647,FOODS_3_810_WI_3_validation,3031,6,9,2,2,d_1913,1.0,2016-04-24,11613,...,0.714286,0.571429,0.750000,0.72,0.571429,0.678571,0.38,0.273973,0.685509,0.399663


In [9]:
#ds=ds2

In [51]:
## 월간 평균 지수 추가
ssn_dt= ds[ds['year'].isin([2011,2012,2013,2014,2015])]

ssn_all= ssn_dt.groupby(['month']).agg({'sales':'mean'}).reset_index().rename({'sales':'ssn_all'},axis=1)
ssn_byitem= ssn_dt.groupby(['month','item_id']).agg({'sales':'mean'}).reset_index().rename({'sales':'ssn_byitem'},axis=1)
ssn_bystore= ssn_dt.groupby(['month','store_id']).agg({'sales':'mean'}).reset_index().rename({'sales':'ssn_bystore'},axis=1)
ssn_bycat= ssn_dt.groupby(['month','cat_id']).agg({'sales':'mean'}).reset_index().rename({'sales':'ssn_bycat'},axis=1)
ssn_bystate= ssn_dt.groupby(['month','state_id']).agg({'sales':'mean'}).reset_index().rename({'sales':'ssn_bystate'},axis=1)

ds1= pd.merge(ds,ssn_all,on=['month'],how='left')
ds1= pd.merge(ds1,ssn_byitem,on=['month','item_id'],how='left')
ds1= pd.merge(ds1,ssn_bystore,on=['month','store_id'],how='left')
ds1= pd.merge(ds1,ssn_bystate,on=['month','state_id'],how='left')
ds1= pd.merge(ds1,ssn_bycat,on=['month','cat_id'],how='left')

In [52]:
ds1['ssn_concat']= ds1['ssn_byitem']*ds1['ssn_bystore']*ds1['ssn_bystate']*ds1['ssn_bycat']

In [53]:
ds1.columns

Index(['id', 'item_id', 'dept_id', 'store_id', 'cat_id', 'state_id', 'd',
       'sales', 'date', 'wm_yr_wk', 'weekday', 'wday', 'month', 'year',
       'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2',
       'snap_CA', 'snap_TX', 'snap_WI', 'sell_price', 'sales_1', 'sales_7',
       'sales_28', 'rmean_1_7', 'rmean_7_7', 'rmean_7_28', 'rmean_7_50',
       'rmean_28_7', 'rmean_28_28', 'rmean_28_50', 'rmean_1_365', 'rstd_365',
       'ssn_all', 'ssn_byitem', 'ssn_bystore', 'ssn_bystate', 'ssn_bycat',
       'ssn_concat'],
      dtype='object')

In [54]:
## 증감율 변수 추가
#ds1['lag_diff_728']= ds1['lag_7']-ds1['lag_28']
#ds1['lag_mean_diff_728']= ds1['rmean_7_28']-ds1['rmean_28_28']
#ds1['lag_mean_diff_77']= ds1['lag_7']-ds1['rmean_7_7']

In [55]:
from random import randrange
from pandas import Series
from matplotlib import pyplot
from statsmodels.tsa.seasonal import seasonal_decompose

In [56]:
#series = [i+randrange(10) for i in range(1,100)]
#result = seasonal_decompose(series, model='additive', freq=1)
#result.plot()
#pyplot.show()

In [57]:
#ds1.groupby(['state_id','store_id','cat_id','item_id']).agg({'sales':'sum'})

In [58]:
ds1.columns

Index(['id', 'item_id', 'dept_id', 'store_id', 'cat_id', 'state_id', 'd',
       'sales', 'date', 'wm_yr_wk', 'weekday', 'wday', 'month', 'year',
       'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2',
       'snap_CA', 'snap_TX', 'snap_WI', 'sell_price', 'sales_1', 'sales_7',
       'sales_28', 'rmean_1_7', 'rmean_7_7', 'rmean_7_28', 'rmean_7_50',
       'rmean_28_7', 'rmean_28_28', 'rmean_28_50', 'rmean_1_365', 'rstd_365',
       'ssn_all', 'ssn_byitem', 'ssn_bystore', 'ssn_bystate', 'ssn_bycat',
       'ssn_concat'],
      dtype='object')

In [59]:
ds1.head()

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,...,rmean_28_28,rmean_28_50,rmean_1_365,rstd_365,ssn_all,ssn_byitem,ssn_bystore,ssn_bystate,ssn_bycat,ssn_concat
0,HOBBIES_1_002_CA_1_validation,1,0,0,0,0,d_250,0.0,2011-10-05,11136,...,NaN,NaN,NaN,NaN,1.368099,0.301325,1.572069,1.522204,0.701315,0.505699
1,HOBBIES_1_002_CA_1_validation,1,0,0,0,0,d_251,0.0,2011-10-06,11136,...,NaN,NaN,NaN,0.000000,1.368099,0.301325,1.572069,1.522204,0.701315,0.505699
2,HOBBIES_1_002_CA_1_validation,1,0,0,0,0,d_252,0.0,2011-10-07,11136,...,NaN,NaN,NaN,0.000000,1.368099,0.301325,1.572069,1.522204,0.701315,0.505699
3,HOBBIES_1_004_CA_1_validation,3,0,0,0,0,d_250,0.0,2011-10-05,11136,...,NaN,NaN,NaN,NaN,1.368099,1.992053,1.572069,1.522204,0.701315,3.343173
4,HOBBIES_1_004_CA_1_validation,3,0,0,0,0,d_251,4.0,2011-10-06,11136,...,NaN,NaN,NaN,2.828427,1.368099,1.992053,1.572069,1.522204,0.701315,3.343173


[M5 아이디어]

1. LSTM
  - seq2seq, transformer , attention -- 영진
  - input 추가 - 1~12, 주, 행사

2. LGBM 3개


3. 2016년 데이터만 학습하니까 RMSE 더 낮더라
4. 주기성, 계절성 반영 ---- 호섭
   - 11~15년 상품별 월 평균 판매량 반매
   - (추가 예정)seasonality decomposition
5. 판매량 scaling -- 호섭, 용선
   - Min-max scaling 
   - Log scaling
   - 0을 -1로 학습하여 나중에 0으로 치환 
6. 이상치 제거  - 호섭, 용선 
   - 3* 표준편차 이상 제거 후 학습
   - 재고 없음 -> 2016년도 학습으로 커버
7. 판매량을 카테고리 변수로 변경
8. decompose를 해서 잔차만 인풋으로 넣고 
sales --> decompe -> 4개 -- 
-> 잔차(상품, state, ~~)만 모델에 Input ,Y : 4개의 값의 -> 예측 단계 1/1 -> 1/1 잔차ㅇ 예측 : -> 3개 는

9. 모델 세분화 - 호섭, 용선
   - state 별 

10. pseudo labeling : 시각화 시켜서 0으로 치환 

11. Input 최근 7, 28일 표준편차 넣기

In [103]:
ds.columns

Index(['id', 'item_id', 'dept_id', 'store_id', 'cat_id', 'state_id', 'd',
       'sales', 'date', 'wm_yr_wk', 'weekday', 'wday', 'month', 'year',
       'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2',
       'snap_CA', 'snap_TX', 'snap_WI', 'sell_price', 'lag_7', 'lag_28',
       'rmean_7_7', 'rmean_28_7', 'rmean_7_28', 'rmean_28_28', 'week',
       'quarter', 'mday'],
      dtype='object')

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
ds1[['sales']]= scaler.fit_transform(ds1[['sales']])

In [104]:
len(ds1)-len(ds)

0

In [105]:
ds2=ds
ds=ds1

In [113]:
ds= ds[ds['year'].isin([2016])]

### 4. 불필요한 데이터와 컬럼 제거

In [114]:
# Remove all rows with NaN value
ds.dropna(inplace = True)

# Define columns that need to be removed
unusedCols = ["id", "date", "sales","d", "wm_yr_wk", "weekday"]
trainCols = ds.columns[~ds.columns.isin(unusedCols)]
X_train = ds[trainCols]
y_train = ds["sales"]

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [115]:
len(ds)

3503543

### 5. 학습, 검증 데이터셋 분리

In [116]:
np.random.seed(120)

# Define categorical features
catFeats = ['item_id', 'dept_id','store_id', 'cat_id', 'state_id'] + \
           ["event_name_1", "event_name_2", "event_type_1", "event_type_2"]

validInds = np.random.choice(X_train.index.values, 2_000_000, replace = False)
trainInds = np.setdiff1d(X_train.index.values, validInds)

trainData = lgb.Dataset(X_train.loc[trainInds], label = y_train.loc[trainInds], 
                        categorical_feature = catFeats, free_raw_data = False)
validData = lgb.Dataset(X_train.loc[validInds], label = y_train.loc[validInds],
                        categorical_feature = catFeats, free_raw_data = False)

In [117]:
#del ds
del X_train, y_train, validInds, trainInds ; gc.collect()

139

### 6. 모델 생성

In [118]:
params = {
          "objective" : "poisson",
          "metric" :"rmse",
          "force_row_wise" : True,
          "learning_rate" : 0.075,
          "sub_row" : 0.75,
          "bagging_freq" : 1,
          "lambda_l2" : 0.1,
          "metric": ["rmse"],
          'verbosity': 1,
          'num_iterations' : 200,
          'num_leaves': 64}

# Train LightGBM model
m_lgb = lgb.train(params, trainData, valid_sets = [validData], verbose_eval = 20) 

# Save the model
m_lgb.save_model("model.lgb")

[20]	valid_0's rmse: 2.50414
[40]	valid_0's rmse: 2.22284
[60]	valid_0's rmse: 2.15293
[80]	valid_0's rmse: 2.13347
[100]	valid_0's rmse: 2.12516
[120]	valid_0's rmse: 2.11933
[140]	valid_0's rmse: 2.11453
[160]	valid_0's rmse: 2.11016
[180]	valid_0's rmse: 2.10653
[200]	valid_0's rmse: 2.10277


### 7. 예측하기

In [119]:
# Last day used for training
trLast = 1913
# Maximum lag day
maxLags = 57

# Create dataset for predictions
def create_ds():
    
    startDay = trLast - maxLags
    
    numCols = [f"d_{day}" for day in range(startDay, trLast + 1)]
    catCols = ['id', 'item_id', 'dept_id','store_id', 'cat_id', 'state_id']
    
    dtype = {numCol:"float32" for numCol in numCols} 
    dtype.update({catCol: "category" for catCol in catCols if catCol != "id"})
    
    ds = pd.read_csv(path+"sales_train_validation.csv", 
                     usecols = catCols + numCols, dtype = dtype)
    
    for col in catCols:
        if col != "id":
            ds[col] = ds[col].cat.codes.astype("int16")
            ds[col] -= ds[col].min()
    
    for day in range(trLast + 1, trLast+ 28 +1):
        ds[f"d_{day}"] = np.nan
    
    ds = pd.melt(ds,
                 id_vars = catCols,
                 value_vars = [col for col in ds.columns if col.startswith("d_")],
                 var_name = "d",
                 value_name = "sales")
    
    ds = ds.merge(calendar, on = "d", copy = False)
    ds = ds.merge(prices, on = ["store_id", "item_id", "wm_yr_wk"], copy = False)
    
    return ds

def create_features(ds):          
    dayLags = [7, 28]
    lagSalesCols = [f"lag_{dayLag}" for dayLag in dayLags]
    for dayLag, lagSalesCol in zip(dayLags, lagSalesCols):
        ds[lagSalesCol] = ds[["id","sales"]].groupby("id")["sales"].shift(dayLag)

    windows = [7, 28]
    for window in windows:
        for dayLag, lagSalesCol in zip(dayLags, lagSalesCols):
            ds[f"rmean_{dayLag}_{window}"] = ds[["id", lagSalesCol]].groupby("id")[lagSalesCol].transform(lambda x: x.rolling(window).mean())
          
    dateFeatures = {"wday": "weekday",
                    "week": "weekofyear",
                    "month": "month",
                    "quarter": "quarter",
                    "year": "year",
                    "mday": "day"}

    for featName, featFunc in dateFeatures.items():
        if featName in ds.columns:
            ds[featName] = ds[featName].astype("int16")
        else:
            ds[featName] = getattr(ds["date"].dt, featFunc).astype("int16")

In [120]:
fday = datetime(2016,4,25) 
alphas = [1.028, 1.023, 1.018]
weights = [1/len(alphas)] * len(alphas)
sub = 0.

for icount, (alpha, weight) in enumerate(zip(alphas, weights)):

    te = create_ds()
    cols = [f"F{i}" for i in range(1,29)]

    for tdelta in range(0, 28):
        day = fday + timedelta(days=tdelta)
        print(tdelta, day)
        tst = te[(te['date'] >= day - timedelta(days=maxLags)) & (te['date'] <= day)].copy()
        create_features(tst)
        tst = tst.loc[tst['date'] == day , trainCols]
        te.loc[te['date'] == day, "sales"] = alpha * m_lgb.predict(tst) # magic multiplier by kyakovlev

    te_sub = te.loc[te['date'] >= fday, ["id", "sales"]].copy()
    te_sub["F"] = [f"F{rank}" for rank in te_sub.groupby("id")["id"].cumcount()+1]
    te_sub = te_sub.set_index(["id", "F" ]).unstack()["sales"][cols].reset_index()
    te_sub.fillna(0., inplace = True)
    te_sub.sort_values("id", inplace = True)
    te_sub.reset_index(drop=True, inplace = True)
    te_sub.to_csv(f"submission_{icount}.csv",index=False)
    if icount == 0 :
        sub = te_sub
        sub[cols] *= weight
    else:
        sub[cols] += te_sub[cols]*weight
    print(icount, alpha, weight)


sub2 = sub.copy()
sub2["id"] = sub2["id"].str.replace("validation$", "evaluation")
sub = pd.concat([sub, sub2], axis=0, sort=False)
#sub.to_csv(path+"submission.csv",index=False)

0 2016-04-25 00:00:00


C:\Users\User\Anaconda3\lib\site-packages\pandas\core\indexing.py:1418: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


1 2016-04-26 00:00:00
2 2016-04-27 00:00:00
3 2016-04-28 00:00:00
4 2016-04-29 00:00:00
5 2016-04-30 00:00:00
6 2016-05-01 00:00:00
7 2016-05-02 00:00:00
8 2016-05-03 00:00:00
9 2016-05-04 00:00:00
10 2016-05-05 00:00:00
11 2016-05-06 00:00:00
12 2016-05-07 00:00:00
13 2016-05-08 00:00:00
14 2016-05-09 00:00:00
15 2016-05-10 00:00:00
16 2016-05-11 00:00:00
17 2016-05-12 00:00:00
18 2016-05-13 00:00:00
19 2016-05-14 00:00:00
20 2016-05-15 00:00:00
21 2016-05-16 00:00:00
22 2016-05-17 00:00:00
23 2016-05-18 00:00:00
24 2016-05-19 00:00:00
25 2016-05-20 00:00:00
26 2016-05-21 00:00:00
27 2016-05-22 00:00:00
0 1.028 0.3333333333333333
0 2016-04-25 00:00:00


C:\Users\User\Anaconda3\lib\site-packages\pandas\core\indexing.py:1418: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


1 2016-04-26 00:00:00
2 2016-04-27 00:00:00
3 2016-04-28 00:00:00
4 2016-04-29 00:00:00
5 2016-04-30 00:00:00
6 2016-05-01 00:00:00
7 2016-05-02 00:00:00
8 2016-05-03 00:00:00
9 2016-05-04 00:00:00
10 2016-05-05 00:00:00
11 2016-05-06 00:00:00
12 2016-05-07 00:00:00
13 2016-05-08 00:00:00
14 2016-05-09 00:00:00
15 2016-05-10 00:00:00
16 2016-05-11 00:00:00
17 2016-05-12 00:00:00
18 2016-05-13 00:00:00
19 2016-05-14 00:00:00
20 2016-05-15 00:00:00
21 2016-05-16 00:00:00
22 2016-05-17 00:00:00
23 2016-05-18 00:00:00
24 2016-05-19 00:00:00
25 2016-05-20 00:00:00
26 2016-05-21 00:00:00
27 2016-05-22 00:00:00
1 1.023 0.3333333333333333
0 2016-04-25 00:00:00


C:\Users\User\Anaconda3\lib\site-packages\pandas\core\indexing.py:1418: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


1 2016-04-26 00:00:00
2 2016-04-27 00:00:00
3 2016-04-28 00:00:00
4 2016-04-29 00:00:00
5 2016-04-30 00:00:00
6 2016-05-01 00:00:00
7 2016-05-02 00:00:00
8 2016-05-03 00:00:00
9 2016-05-04 00:00:00
10 2016-05-05 00:00:00
11 2016-05-06 00:00:00
12 2016-05-07 00:00:00
13 2016-05-08 00:00:00
14 2016-05-09 00:00:00
15 2016-05-10 00:00:00


KeyboardInterrupt: 

In [ ]:
sub.to_csv(path+"hello_submission.csv",index=False)

In [119]:
test_dt= pd.read_csv("C:/Users/User/Desktop/results/submission.csv")

In [120]:
test_dt.loc[test_dt['id'].isin(zero_fnl),'F1']=0.0
test_dt.loc[test_dt['id'].isin(zero_fnl),'F2']=0.0
test_dt.loc[test_dt['id'].isin(zero_fnl),'F3']=0.0
test_dt.loc[test_dt['id'].isin(zero_fnl),'F4']=0.0
test_dt.loc[test_dt['id'].isin(zero_fnl),'F5']=0.0
test_dt.loc[test_dt['id'].isin(zero_fnl),'F6']=0.0
test_dt.loc[test_dt['id'].isin(zero_fnl),'F7']=0.0
test_dt.loc[test_dt['id'].isin(zero_fnl),'F8']=0.0
test_dt.loc[test_dt['id'].isin(zero_fnl),'F9']=0.0
test_dt.loc[test_dt['id'].isin(zero_fnl),'F10']=0.0
test_dt.loc[test_dt['id'].isin(zero_fnl),'F11']=0.0
test_dt.loc[test_dt['id'].isin(zero_fnl),'F12']=0.0
test_dt.loc[test_dt['id'].isin(zero_fnl),'F13']=0.0
test_dt.loc[test_dt['id'].isin(zero_fnl),'F14']=0.0
test_dt.loc[test_dt['id'].isin(zero_fnl),'F15']=0.0
test_dt.loc[test_dt['id'].isin(zero_fnl),'F16']=0.0
test_dt.loc[test_dt['id'].isin(zero_fnl),'F17']=0.0
test_dt.loc[test_dt['id'].isin(zero_fnl),'F18']=0.0
test_dt.loc[test_dt['id'].isin(zero_fnl),'F19']=0.0
test_dt.loc[test_dt['id'].isin(zero_fnl),'F20']=0.0
test_dt.loc[test_dt['id'].isin(zero_fnl),'F21']=0.0
test_dt.loc[test_dt['id'].isin(zero_fnl),'F22']=0.0
test_dt.loc[test_dt['id'].isin(zero_fnl),'F23']=0.0
test_dt.loc[test_dt['id'].isin(zero_fnl),'F24']=0.0
test_dt.loc[test_dt['id'].isin(zero_fnl),'F25']=0.0
test_dt.loc[test_dt['id'].isin(zero_fnl),'F26']=0.0
test_dt.loc[test_dt['id'].isin(zero_fnl),'F27']=0.0
test_dt.loc[test_dt['id'].isin(zero_fnl),'F28']=0.0

In [133]:
test_dt.to_csv("submission_zero_v5.csv")

In [134]:
test_dt

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,FOODS_1_001_CA_1_validation,1.015899,0.922328,0.903956,0.866628,1.092602,1.291176,1.288363,1.065590,1.038293,...,1.079103,1.378929,1.267483,0.996211,0.922700,0.907568,0.907116,1.047463,1.336893,1.326170
1,FOODS_1_001_CA_2_validation,1.150870,1.164748,1.080786,1.419951,1.378518,1.718000,1.738500,1.048212,1.033828,...,1.325030,1.823450,1.648830,1.241223,1.151209,1.135630,1.189726,1.381668,1.857860,1.499063
2,FOODS_1_001_CA_3_validation,1.147929,1.104814,1.098739,1.077165,1.149750,1.244023,1.047502,0.957887,0.979535,...,1.160246,1.380837,1.375333,1.096882,1.049864,1.064662,1.065184,1.139512,1.345131,1.142945
3,FOODS_1_001_CA_4_validation,0.411316,0.374688,0.362026,0.362026,0.432828,0.470118,0.524625,0.389064,0.420269,...,0.426875,0.482771,0.515643,0.382084,0.387154,0.384701,0.382290,0.429290,0.499526,0.525368
4,FOODS_1_001_TX_1_validation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60975,HOUSEHOLD_2_516_TX_2_evaluation,0.268340,0.251854,0.282073,0.269981,0.294697,0.347167,0.346835,0.266733,0.286179,...,0.265979,0.367641,0.369064,0.258798,0.259008,0.272012,0.258867,0.270993,0.341211,0.312939
60976,HOUSEHOLD_2_516_TX_3_evaluation,0.148936,0.143692,0.155443,0.146295,0.158409,0.174848,0.170126,0.147947,0.138210,...,0.152113,0.167010,0.167431,0.150043,0.145889,0.146400,0.146400,0.157264,0.172971,0.173305
60977,HOUSEHOLD_2_516_WI_1_evaluation,0.107775,0.105202,0.105831,0.105831,0.092618,0.103431,0.105994,0.126196,0.124501,...,0.130277,0.156798,0.159144,0.125677,0.122883,0.123040,0.123040,0.133107,0.151529,0.159774
60978,HOUSEHOLD_2_516_WI_2_evaluation,0.054256,0.053467,0.053629,0.116560,0.115487,0.116679,0.124696,0.123739,0.121670,...,0.146898,0.156566,0.162373,0.142225,0.139451,0.139737,0.120109,0.130833,0.139450,0.150329
